In [28]:
using BlossomV, LinearAlgebra, SparseArrays

┌ Info: Precompiling BlossomV [6c721016-9dae-5d90-abf6-67daaccb2332]
└ @ Base loading.jl:1278
ERROR: LoadError: BlossomV not properly installed. Please run Pkg.build("BlossomV")
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] top-level scope at C:\Users\Stefano\.julia\packages\BlossomV\szW7L\src\BlossomV.jl:6
 [3] include(::Function, ::Module, ::String) at .\Base.jl:380
 [4] include(::Module, ::String) at .\Base.jl:368
 [5] top-level scope at none:2
 [6] eval at .\boot.jl:331 [inlined]
 [7] eval(::Expr) at .\client.jl:467
 [8] top-level scope at .\none:3
in expression starting at C:\Users\Stefano\.julia\packages\BlossomV\szW7L\src\BlossomV.jl:3


LoadError: Failed to precompile BlossomV [6c721016-9dae-5d90-abf6-67daaccb2332] to C:\Users\Stefano\.julia\compiled\v1.5\BlossomV\akfpb_dRERM.ji.

In [29]:
import Pkg; Pkg.build("BlossomV")

   Building BlossomV → `C:\Users\Stefano\.julia\packages\BlossomV\szW7L\deps\build.log`
┌ Error: Error building `BlossomV`: 
│ ERROR: LoadError: None of the selected providers can install dependency blossom5int32.
│ Use BinDeps.debug(package_name) to see available providers
│ 
│ Stacktrace:
│  [1] error(::String) at .\error.jl:33
│  [2] satisfy!(::BinDeps.LibraryDependency, ::Array{DataType,1}) at C:\Users\Stefano\.julia\packages\BinDeps\5xsHU\src\dependencies.jl:956
│  [3] satisfy!(::BinDeps.LibraryDependency) at C:\Users\Stefano\.julia\packages\BinDeps\5xsHU\src\dependencies.jl:929
│  [4] top-level scope at C:\Users\Stefano\.julia\packages\BinDeps\5xsHU\src\dependencies.jl:984
│  [5] include(::String) at .\client.jl:457
│  [6] top-level scope at none:5
│ in expression starting at C:\Users\Stefano\.julia\packages\BlossomV\szW7L\deps\build.jl:72
└ @ Pkg.Operations D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Pkg\src\Operations.jl:949


In [10]:
# Implements algorithm 1 in Improved Algorithms for Detecting Negative Cost Cycles in Undirected Graphs.
# Xiaofeng Gu1, Kamesh Madduri, K. Subramani and Hong-Jian Lai

function optimal_cycle(G)
    G = sparse(G)
    rows,weights  = rowvals(G), nonzeros(G)
    ∂(i) = zip((@view rows[nzrange(G,i)]), (@view weights[nzrange(G,i)]))
    n=size(G,1);
    m=Int(length(rows)/2)
    E=Matching(Float64, 2n+2m)
    k = 2n+1
    edge(i,j,w) = add_edge(E, i-1, j-1, w)
    D=Dict{Int, Tuple{Int,Int}}()
    for i=1:n
        edge(i, i+n, 0)
        for (j,w) in ∂(i)
            if i < j
                edge(i,     k,   w)
                edge(i + n, k,   w)
                edge(j,     k+1, w)
                edge(j + n, k+1, w)
                edge(k,     k+1, 0)
                D[k]=(i,j)
                k+=2
            end
        end
    end
    @assert k == 2n+2m+1

    solve(E)

    sol=Tuple{Int,Int}[]
    for k=2n:2:2n+2m-1
        if get_match(E, k) != k + 1
            push!(sol, D[k+1])
        end
    end

    sol
end

optimal_cycle (generic function with 1 method)

In [11]:
#edge list, indices start at 0

G = [0  1  1  0  0 0;
     1  0 -3 -1  0 0;
     1 -3  0  0 -2 0;
     0 -1  0  0  0 0;
     0  0 -2  0  0 1;
     0  0  0  0  1 0]

@assert(issymmetric(G))

LoadError: UndefVarError: issymmetric not defined

In [21]:
op = optimal_cycle(G)

LoadError: UndefVarError: sparse not defined

In [14]:
using LightGraphs, SimpleWeightedGraphs, GraphRecipes, Plots

In [12]:
g = SimpleWeightedGraph(G)

{6, 6} undirected simple Int64 graph with Int64 weights

In [33]:
ec = [:black for i in G[:]]
graphplot(g, edgelabel=weights(g), names=vertices(g), nodecolor=:yellow, edgecolor=ec)

LoadError: BoundsError: attempt to access 36-element Array{Symbol,1} at index [1, 2]

In [35]:
ec = [:black for i in CartesianIndices(G)]

6×6 Array{Symbol,2}:
 :black  :black  :black  :black  :black  :black
 :black  :black  :black  :black  :black  :black
 :black  :black  :black  :black  :black  :black
 :black  :black  :black  :black  :black  :black
 :black  :black  :black  :black  :black  :black
 :black  :black  :black  :black  :black  :black